In [1]:
import itertools

import pandas as pd

import numpy as np

import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data_df = pd.read_csv("~/Downloads/tasks.csv")
data_dict = data_df.to_dict()

betas_df = pd.read_csv("~/Downloads/betas.csv")

d = dict.fromkeys(betas_df.cluster, [])
for k, v in zip(betas_df.cluster, betas_df.task):
    d[k] = d[k] +[v]

In [3]:
def get_key(my_dict, val):
    for k, v in my_dict.items():
         if val in v:
             return k
    return "There is no such key"

def subset_data(data_dict, key_value, key_name = "task", test_size = 0.33):
    if type(data_dict[key_name]) == list:
        values = data_dict[key_name]
    else:
        values = list(data_dict[key_name].values())
    idx_task = np.where(np.array(values) == key_value)
    idx_task = idx_task[0].tolist()
    x = [data_dict['x'][i] for i in idx_task]
    X = np.array([np.ones(len(idx_task)), np.array(x)]).T
    y = np.array([data_dict['y'][i] for i in idx_task])
    if test_size == 0:
        return X, y
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size = test_size,
                                                        random_state = 123)
    return X_train, X_test, y_train, y_test

def mse(model, X_true, y_true):
    y_predict = model.predict(X_true)
    mse = np.mean((y_predict - y_true) ** 2)
    return mse

def prepare_input(data_dict, target_task):
    input_data = {"data_dict": data_dict}
    input_data["X_target_train"], input_data["X_target_test"], input_data["y_target_train"], input_data["y_target_test"] = subset_data(data_dict, key_value = target_task, key_name = "task")
    input_data["X_target_val"], input_data["X_target_test"], input_data["y_target_val"], input_data["y_target_test"] = train_test_split(input_data["X_target_test"], input_data["y_target_test"], 
                                                        test_size = 0.5,
                                                        random_state = 123)
    
    input_data["source_task"] = list(set(list(itertools.chain.from_iterable(d.values()))) - set([target_task]))
    
    source_cluster = [get_key(d, i) for i in input_data["source_task"]]
    input_data["source_cluster"] = list(set(source_cluster))
    
    idx_source = np.where(np.array(list(data_dict['task'].values())) != target_task)[0].tolist()
    
    # source data
    input_data["source_dict"] = {}
    for key_name in data_dict.keys():
        input_data["source_dict"][key_name] = [data_dict[key_name][i] for i in idx_source]

    
    return(input_data)

In [4]:
# add key "cluster" to `data_dict`
data_dict["cluster"] = []

for task in data_dict["task"].values():
    cluster = get_key(d, task)
    if(cluster == "There is no such key"):
        print("task = " + str(task))
        break
    data_dict["cluster"].append(cluster)

In [5]:
def get_bandit(alpha, beta, t, pi):
    source_cluster = alpha.keys()
    for cluster in source_cluster:
        if t == 0:
            pi[cluster] = [np.random.beta(alpha[cluster][t], beta[cluster][t])]
        else:
            pi[cluster].append(np.random.beta(alpha[cluster][t], beta[cluster][t]))
    pi_list = [pi[cluster][t] for cluster in input_data["source_cluster"]]
    bandit = get_key(pi, max(pi_list))
    return(bandit, pi)

In [6]:
def update_hyper_para(alpha, beta, t, evaluation, bandit_current):
    # for selected bandits
    if evaluation[-1] > evaluation[-2]:
        alpha[bandit_current] = alpha[bandit_current] + [alpha[bandit_current][t] + 2]
        beta[bandit_current] = beta[bandit_current] + [beta[bandit_current][t]]
    else:
        alpha[bandit_current]  = alpha[bandit_current] + [alpha[bandit_current][t]]
        beta[bandit_current] = beta[bandit_current] + [beta[bandit_current][t] + 2]
    # for unselected bandits
    for bandit in alpha.keys():
        if len(alpha[bandit]) < t + 2:
           alpha[bandit] = alpha[bandit] + [alpha[bandit][t]]
           beta[bandit] = beta[bandit] + [beta[bandit][t]]
    return alpha, beta

In [7]:
def bandit_source_train(input_data, model, batch_size, decay_rate, n_it, metric):
    
    # initialize hyperparameters
    alpha = beta = dict.fromkeys(input_data["source_cluster"], [1])
    pi = dict.fromkeys(input_data["source_cluster"], [0])
    
    # initialize model from target training data
    mod_train = model.fit(input_data["X_target_train"], input_data["y_target_train"])
    evaluation = [metric(mod_train, input_data["X_target_val"], input_data["y_target_val"])]
    
    
    for t in range(n_it):
        # select bandit
        bandit_current, pi = get_bandit(alpha, beta,t, pi)
        
        # set training data at this iteration
        X_current, y_current = subset_data(input_data["source_dict"], 
                                   key_value = bandit_current,
                                   key_name = "cluster", test_size = 0)
        batch_id = random.choices(list(range(0, len(y_current))), k = batch_size)
        X_current, y_current = X_current[batch_id, :], y_current[batch_id]
        
        X_current = np.concatenate((X_current, input_data["X_target_val"]), axis = 0)
        y_current = np.concatenate((y_current, input_data["y_target_val"]), axis = 0)
        
        # train model
        mod_train = model.fit(X_current, y_current)
        # evaluate model
        evaluation += [metric(mod_train, input_data["X_target_val"], input_data["y_target_val"])]
        
        # update bandit parameters 
        alpha, beta = update_hyper_para(alpha, beta, t, evaluation, bandit_current)
    
    return evaluation, alpha, beta

In [8]:
input_data = prepare_input(data_dict, target_task = 2)
evaluation, alpha, beta = bandit_source_train(input_data, model = LinearRegression(), batch_size = 8,
                    decay_rate = 0, n_it = 100, metric = mse)

In [9]:
evaluation

[0.07329569589006937,
 0.661559497562852,
 0.16492672621732368,
 0.37729308559069175,
 0.19164037142887957,
 0.4520628851400674,
 0.09775433010990082,
 0.12876643954070835,
 1.8406384401196885,
 0.4297352272947138,
 0.11424165763486553,
 0.6461847135329928,
 0.17562556574691093,
 0.6049100074389122,
 0.24957793281476082,
 0.5046525398499419,
 0.10911665862241182,
 1.6357079867436575,
 0.46670543954002824,
 2.1158770173234682,
 0.48235625331241355,
 0.20390122743512026,
 0.4375670274031541,
 1.950521454470534,
 0.5937803699743096,
 1.7868417354896975,
 0.5275034142432358,
 2.2476061595897665,
 0.4877527822146553,
 0.2877082464815802,
 0.5961897605001208,
 0.15139721486879804,
 0.11262050275353747,
 0.5887871008176742,
 0.22822539914838808,
 0.1514076845305178,
 0.505142684247735,
 2.2092590752073944,
 0.14425967024965086,
 0.21849121381443515,
 1.3854263203675599,
 2.496578552018015,
 0.20980660617181338,
 0.3725136943197989,
 0.5218551250227232,
 1.4160590129593285,
 0.0870661880310962